In [1]:
pip install google

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install numpy
!pip install scikit-surprise
import os
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.accuracy import rmse
from collections import defaultdict
from surprise import KNNBasic
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import accuracy
from surprise import AlgoBase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 10.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633961 sha256=34dcb5155d901ec09a69c09850446ffbf47e405bf06c1de4add3ba2b26f0ec5a
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [4]:

# Function to load the ratings and movies datasets

dir = '/content/drive/MyDrive/CSE573-SWM-Movie-Recommender/DATA'
def loadDataset():
  ratings_df = pd.read_csv(dir+'/ratings.csv')
  movies_df = pd.read_csv(dir+'/movies.csv')
  
  ratings_df.drop('timestamp',axis =1, inplace = True)

  return ratings_df,movies_df


In [5]:
# Calculates the evaluations metrics Precision and Recall for each loop of cross validation

def metricsAtK(predictions, k=10, threshold=3.5):

    # dictionary to store user and the respective predicted ratings of the movies
    user_predicted = defaultdict(list)
    for u_id, _, rating_actual, rating_predicted, _ in predictions:
        user_predicted[u_id].append((rating_predicted, rating_actual))

    precisions = dict()
    recalls = dict()

    for u_id, u_ratings in user_predicted.items():
        u_ratings.sort(key=lambda x: x[0], reverse=True)

        # Calculating actual values of the ratings
        actual = sum((ratings_true >= threshold) for (_, ratings_true) in u_ratings)
        # Calculating predicted values of the ratings
        predicted = sum((estimate_value >= threshold) for (estimate_value, _) in u_ratings[:k])

        # Calculating true positives and negatives
        positive_true = sum(((ratings_true >= threshold) and (estimate_value >= threshold))
                              for (estimate_value, ratings_true) in u_ratings[:k])
        negative_true = sum(((ratings_true < threshold) and (estimate_value < threshold))
                              for (estimate_value, ratings_true) in u_ratings[:k])
        
        # Calculating actual and predicted positives and negatives
        positive_predicted = predicted if predicted != 0 else 1
        negative_predicted = predicted if predicted != 1 else 0
        positive_actual = actual if actual != 0 else 1
        negative_actual = actual if actual != 1 else 0

        # Calculating precision, recall and accuracy
        precisions[u_id] = positive_true / positive_predicted
        recalls[u_id] = positive_true / positive_actual

    return precisions, recalls

In [6]:
def fit(model, ratings_data, num_of_splits=5):
  # Generator object for k - cross validation
  kf = KFold(n_splits=num_of_splits)
  split_df = list()

  i = 1
  for train, test in kf.split(ratings_data):
      predictions = model.fit(train).test(test)
      rmse = accuracy.rmse(predictions, verbose=False)
      mae = accuracy.mae(predictions, verbose=False)
      precisions, recalls = metricsAtK(predictions, k=5, threshold=4)
      precision = sum(prec for prec in precisions.values()) / len(precisions)
      recall = sum(rec for rec in recalls.values()) / len(recalls)
      f1_score = (2*precision*recall)/(precision+recall)
      split_df.append([i,precision,recall,rmse,f1_score,mae])
      i +=1

  split_df = pd.DataFrame(split_df, columns=['Split', 'Precision', 'Recall', 'RMSE', 'F1 Score', 'MAE'])
  return split_df

In [7]:
def getSortedPredictions(predictions):
    
    sorted_predictions = defaultdict(list)    
    for u_id, id, _, rating_predicted, _ in predictions:
        sorted_predictions[u_id].append((id, rating_predicted))

    for u_id, u_ratings in sorted_predictions.items():
        u_ratings.sort(key=lambda x: x[1], reverse=True)

    return sorted_predictions

In [8]:
def inference(model):
  trainset = ratings_data.build_full_trainset()

  # build_anti_testset will generate the entires for the movies which the user has not rated. i.e 
  # The completement of the user's ratings. It assumes the rating to be equal to the global mean of the ratings.
  testset = trainset.build_anti_testset() 

  # Training the model with trainset and getting predictions using the generated testset
  model.fit(trainset)
  predictions = model.test(testset)

  return predictions

In [9]:
# Post processing the predictions to produce the list of recommendations
def getRecommendations(predictions, n=10):

  # Getting sorted predictions
  total = getSortedPredictions(predictions)

  # Extracting only n number of movie predictions
  for user_id, user_ratings in total.items():
      total[user_id] = user_ratings[:n]

  total_df = pd.DataFrame.from_dict(total)
  total_df = total_df.transpose()

  result = []
  for user_id,user_ratings in total.items():
    result.append(total_df.loc[user_id])

  #Developing recommendations
  recommendations = []
  for i in result:
    recommended_movieIds=[]
    for x in range(0, n):
      recommended_movieIds.append(i[x][0])
    recommendations.append(recommended_movieIds)

  recommendation_list = []
  for i in recommendations:
    df = movies_df[movies_df['movieId'].isin(i)]
    temp = df['title'].tolist()
    recommendation_list.append(temp)

  recommendation_df = pd.DataFrame(recommendation_list)
  return recommendation_df

## Loading Dataset

In [10]:
ratings_df , movies_df = loadDataset()

In [11]:
reader = Reader(rating_scale=(0.5, 5.0))
ratings_keys = ['userId', 'movieId', 'rating']
ratings_filter = ratings_df[ratings_keys]
ratings_data = Dataset.load_from_df(ratings_filter, reader)

## Modeling Selection

## SVD

In [14]:
msd_knn = SVD(n_factors=30, n_epochs=20, lr_all=0.008, reg_all=0.08)
#msd_preds = msd_knn.fit(trainset).test(testset)


In [15]:
# Generating evaluation metrics Precision, Recall, F1-Score, RMSE, MAE for the ratings dataset
msd_metrics = fit(msd_knn, ratings_data)
msd_metrics

,Split,Precision,Recall,RMSE,F1 Score,MAE
0,1,0.548520,0.236316,0.862594,0.330321,0.664543
1,2,0.561349,0.224143,0.863474,0.320366,0.662147
2,3,0.565191,0.231533,0.868780,0.328496,0.668859
3,4,0.544781,0.220180,0.866797,0.313611,0.666397
4,5,0.546721,0.221236,0.866124,0.315003,0.663583


In [16]:
msd_predictions = inference(msd_knn)

In [17]:
msd_df = getRecommendations(msd_predictions)
msd_df.to_csv(dir+'/msd_knn.csv',index = False)

### Using Pearson Correlation

In [18]:
# Defining the similarity options with pearson correlation.
sim_options = {
    'name': 'pearson'
}

pearson_knn = KNNBasic(k= 35, n_epochs=25,sim_options = sim_options)
#cosine_preds = cosine_knn.fit(trainset).test(testset)


In [19]:
# Generating evaluation metrics Precision, Recall, F1-Score, RMSE, MAE for the ratings dataset
pearson_metrics = fit(pearson_knn, ratings_data)
pearson_metrics

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


,Split,Precision,Recall,RMSE,F1 Score,MAE
0,1,0.674863,0.258725,0.977619,0.374049,0.753428
1,2,0.667350,0.252646,0.974418,0.366530,0.751248
2,3,0.665246,0.250244,0.974457,0.363683,0.751480
3,4,0.656557,0.252257,0.962989,0.364478,0.747752
4,5,0.658169,0.256541,0.975063,0.369182,0.751986


In [20]:

pearson_predictions = inference(pearson_knn)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [22]:

pearson_df = getRecommendations(pearson_predictions)
pearson_df.to_csv(dir+'/pearson_knn.csv',index = False)

### Using Cosine similarity

In [21]:
# Defining the similarity options with cosine similarity.
sim_options = {
    'name': 'cosine'
}

cosine_knn = KNNBasic(k= 35, n_epochs=25)
#cosine_preds = cosine_knn.fit(trainset).test(testset)


In [23]:
# Generating evaluation metrics Precision, Recall, F1-Score, RMSE, MAE for the ratings dataset
cosine_metrics = fit(cosine_knn, ratings_data)
cosine_metrics

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,Split,Precision,Recall,RMSE,F1 Score,MAE
0,1,0.680410,0.270976,0.950086,0.387592,0.726868
1,2,0.671120,0.263702,0.955591,0.378630,0.733790
2,3,0.667679,0.270994,0.941897,0.385516,0.723044
3,4,0.668966,0.267299,0.939538,0.381973,0.718660
4,5,0.662459,0.266613,0.943613,0.380207,0.721111


In [24]:
cosine_predictions = inference(cosine_knn)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [25]:
cosine_df = getRecommendations(cosine_predictions)
cosine_df.to_csv(dir+'/cosine_knn.csv',index = False)
